In [5]:
import pandas as pd
import zipfile
import seaborn as sns # For plotting
import matplotlib.pyplot as plt # For showing plots
import numpy as np
import pandas as pd
import scipy.stats as sts
import scikit_posthocs as sp
import seaborn as sns
import matplotlib.pyplot as plt


In [6]:
df = pd.read_csv('EFIplus_medit.zip',compression='zip', sep=";")

In [7]:
df2 = df.dropna(how='all') # drops rows when all elements are missing values
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5011 entries, 0 to 5010
Columns: 164 entries, Site_code to Iberochondrostoma_sp
dtypes: float64(38), int64(120), object(6)
memory usage: 6.3+ MB


In [8]:
df2 = df.dropna() # drops rows when at least one element is a missing value
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2822 entries, 18 to 5010
Columns: 164 entries, Site_code to Iberochondrostoma_sp
dtypes: float64(38), int64(120), object(6)
memory usage: 3.6+ MB


,Site_code,Latitude,Longitude,Country,Catchment_name,Galiza,Subsample,Calib_EFI_Medit,Calib_connect,Calib_hydrol,...,Squalius malacitanus,Squalius pyrenaicus,Squalius torgalensis,Thymallus thymallus,Tinca tinca,Zingel asper,Squalius sp,Barbatula sp,Phoxinus sp,Iberochondrostoma_sp
18,ES_03_0001,41.957268,-2.926857,Spain,Douro,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
19,ES_03_0002,41.850818,-2.591150,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
28,ES_03_0012,41.498026,-5.528753,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
32,ES_03_0016,41.586853,-3.072064,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
43,ES_03_0029,41.830177,-3.782420,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
import pandas as pd

# Crear la tabla de frecuencia cruzada
country_count = pd.crosstab(index=df2['Country'], columns='count')

# Ordenar el DataFrame por la columna 'count' en orden descendente
country_count_sorted = country_count.sort_values(by='count', ascending=False)

# Mostrar el DataFrame ordenado
print(country_count_sorted)

col_0     count
Country        
Spain      1905
Portugal    866
Italy        51


In [10]:
import pandas as pd
import scipy.stats as sts


data = pd.crosstab(df2['Country'], df2['Salmo trutta fario'])

data.head()

Salmo trutta fario,0,1
Country,,
Italy,26,25
Portugal,614,252
Spain,140,1765


In [24]:
import pandas as pd

# Lee el conjunto de datos
data = df2  # Reemplaza "tu_archivo.csv" con el nombre de tu archivo

# Calcula el número de muestras por cuenca
samples_per_catchment = data['Catchment_name'].value_counts()

# Selecciona las ocho cuencas más muestreadas
eight_most_sampled_catchments = samples_per_catchment.nlargest(8).index.tolist()

print(f'Las ocho cuencas más muestreadas son: {eight_most_sampled_catchments}')

Las ocho cuencas más muestreadas son: ['Galiza-Norte', 'Minho', 'Cantabrica', 'Douro', 'Tejo', 'Guadia', 'Galiza-Sul', 'Mondego']


In [27]:
from scipy.stats import kruskal

# Extract the elevation data for the eight most sampled catchments
catchment_elevation_data = []

for catchment in eight_most_sampled_catchments:
    elevation_data =df2[df2['Catchment_name'] == catchment]['Altitude']
    catchment_elevation_data.append(elevation_data)

# Perform Kruskal-Wallis test
kw_statistic, p_value = kruskal(*catchment_elevation_data)

print("Kruskal-Wallis Test:")
print("H statistic:", kw_statistic)
print("p-value:", p_value)


#no hay diferencias en la elevación media entre las ocho cuencas más muestreadas.

Kruskal-Wallis Test:
H statistic: 725.6716048145258
p-value: 2.0095865138939658e-152


In [9]:

catchment_count = pd.crosstab(index = df2['Catchment_name'], columns='count')
catchment_count.head()

col_0,count
Catchment_name,
Algarve-Sotavento,6
Algarve-barlavento,10
Campania-South,5
Cantabrica,345
Catala,2


In [28]:
presence = df2[df2['Salmo trutta fario'] == 1]
absence = df2[df2['Salmo trutta fario'] == 0]
presence= presence['Altitude'].dropna()
absence = absence['Altitude'].dropna()

# Compute the test
stat, p = sts.kruskal(presence, absence)
print('F-statistics=%.3f, p=%.6f' % (stat, p))
alpha=0.05
if p > alpha:
 print('Fail to reject H0. Rejecting H0 has an error probability >0.05')
else:
 print('Reject H0 with an error probability <0.05)')

F-statistics=250.031, p=0.000000
Reject H0 with an error probability <0.05)


## Wine secction

In [29]:
import pandas as pd
from scipy.stats import f_oneway


data = pd.read_csv('winequality_red.csv')

# 2. Crear una nueva columna para categorizar la calidad del vino
data['quality_category'] = data['quality'].apply(lambda x: 'good' if x > 5 else 'bad')

# 3. Realizar ANOVA para cada parámetro del vino
parameters = data.columns[:-2]  # Excluir 'quality' y 'quality_category'
significant_parameters = []

for param in parameters:
    good_wine = data[data['quality_category'] == 'good'][param]
    bad_wine = data[data['quality_category'] == 'bad'][param]
    
    # Realizar ANOVA
    f_statistic, p_value = f_oneway(good_wine, bad_wine)
    
    # Evaluar significancia (usando un valor de corte de 0.05)
    if p_value < 0.05:
        significant_parameters.append(param)
        print(f"{param}: p-value = {p_value:.4f} (Significant)")

# 4. Mostrar los parámetros del vino que discriminan mejor entre las dos clases de calidad
print("\nParámetros del vino que discriminan mejor entre las dos clases de calidad:")
print(significant_parameters)


fixed acidity: p-value = 0.0001 (Significant)
volatile acidity: p-value = 0.0000 (Significant)
citric acid: p-value = 0.0000 (Significant)
chlorides: p-value = 0.0000 (Significant)
free sulfur dioxide: p-value = 0.0135 (Significant)
total sulfur dioxide: p-value = 0.0000 (Significant)
density: p-value = 0.0000 (Significant)
sulphates: p-value = 0.0000 (Significant)
alcohol: p-value = 0.0000 (Significant)

Parámetros del vino que discriminan mejor entre las dos clases de calidad:
['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'sulphates', 'alcohol']
